In [8]:
%matplotlib widget
from essential_func import *
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
os.chdir('/Users/bashit.a/Documents/Alzheimer/Dec-15/')     # location where your h5 and exp file are

In [9]:
## Load file before WAXS thresholding (Requires to run the folliwing cell only) - should not take more than 3 min to load for M1 macbook

file = '2048_B8.h5'
with h5py.File(file,'r') as hdf:
    dset_waxs = np.array(hdf.get(f'{h5_top_group(file)}/primary/data/pilW2_image'))         # waxs data read from h5 file
    dset_waxs_sum = np.sum(dset_waxs,axis=0)                                # summing all the frame values
    print('Pixalated Sum WAXS shape = ', dset_waxs_sum.shape , 'Mean = {:02f}'.format(dset_waxs_sum.mean()) ,'Min. = {:02f}'.format(dset_waxs_sum.min()), 'Max. = {:02f}'.format(dset_waxs_sum.max()))

Pixalated Sum WAXS shape =  (1043, 981) Mean = 13452.854182 Min. = -9882.000000 Max. = 5767807176.000000


In [10]:
## WAXS thresholding and loading
## specs/Slider Properties
AMIN_DEFAULT_VALUE = 0       # minimum value for the pixalated sum
AMAX_DEFAULT_VALUE = dset_waxs_sum.mean() ; AMAX_MIN_VALUE = dset_waxs_sum.mean() ; AMAX_MAX_VALUE = 60000;  AMAX_SLIDER_STEP = 100;    # maximum values for the pixalated sum
# valid_range_min, valid_range_max = (0,10)   # just for WAXS Display purpose


def waxs_thresholding(a_min, a_max, thr, frame):
    %matplotlib widget
    
    args = (dset_waxs_sum, a_min, a_max, thr)
    dset_waxs_thr, gray_img, thr_fr_img = threshold_patch_one_frame(dset_waxs[frame], args = args)
    
    #### plot thresholded pixalated sum
    f, axs = plt.subplots(nrows=2,ncols=2,figsize=(12,8),num=f'{file}')
    im = axs[0,0].imshow(dset_waxs_thr, cmap = 'rainbow')       #plt.colorbar(im, fraction=0.046, pad=0.04)  
    show_colorbar(im,f,axs[0,0])
    axs[0,0].set_title('Pixelated Sum')
    #### plot global thresholding  
    im = axs[0,1].imshow(gray_img, cmap='gray')       
    axs[0,1].set_title(f'Binary Threshold = {thr}')

    patching(file, frame, qgrid2, args=args, axes = (f,[axs[1,0], axs[1,1]]), method = 'thresholding',  )
    
    print('a_min = ', a_min, 'a_max = {:0.2f}'.format(a_max), 'thr = ', thr)

frame = ipywidgets.IntSlider(value=0, min=0, max=len(dset_waxs), step=1, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
a_min = ipywidgets.fixed(value=AMIN_DEFAULT_VALUE)
a_max = ipywidgets.FloatSlider(value=AMAX_DEFAULT_VALUE, min=AMAX_MIN_VALUE,     max=AMAX_MAX_VALUE, step=AMAX_SLIDER_STEP, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
thr   = ipywidgets.IntSlider(value=a_min.value,          min=AMIN_DEFAULT_VALUE, max=AMAX_MAX_VALUE, step=AMAX_SLIDER_STEP, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
ipywidgets.jslink((a_max,'value'),(thr,'max'))           # make sure threshold's maximum value doesn't exceed a_max value

ipywidgets.interactive(waxs_thresholding, a_min=a_min, a_max=a_max, thr=thr, frame=frame)

interactive(children=(FloatSlider(value=13452.854181510053, continuous_update=False, description='a_max', layo…

In [12]:
######## ---- Test performance of thresholded vs original 1-D plot ---- ######## (parameters depends on 3rd cell)
%matplotlib widget
file = '2048_B8.h5'

Iq_S = read_Iq(file,'_WAXS2')
Width, Height = width_height(file)    # considering the fact that width and height is same for source and masked file
n_patterns = Width*Height 

f,ax = plt.subplots()
def update_plot(frame):
    ax.clear()
    # source plot
    ax.plot(qgrid2, Iq_S[frame], label = f'Source - {frame}')
    
    # masked frame
    args = dset_waxs_sum, a_min.value, a_max.value, thr.value
    Iq_M = patching(file, frame, qgrid2, args=args, method = 'thresholding')
    ax.plot(qgrid2, Iq_M, label = f'Masked - {frame}')
    
    ax.set(xlabel = "$q (\AA^{-1})$", ylabel = "$I$", xscale='linear', yscale = 'log' ); 
    ax.legend()

frame = ipywidgets.IntSlider(min=0, max=n_patterns-1, value=0, step=1, description = "Frame", continuous_update=False, layout=ipywidgets.Layout(width='50%'))
ipywidgets.interactive(update_plot, frame=frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, continuous_update=False, description='Frame', layout=Layout(width='50…

In [14]:
##### --------- 1-d averaging after thresholding for one file  --------- ##### (Independent Cell)
### spec
source_file = "2048_B8.h5"
args =  0, 27152, 25200   # manually can set args = (a_min.value, a_max.value, thr.value) = 0, 30000, 16000 or Get values from 3rd cell print output

### computation
masked_file = circ_avg_from_patches(source_file, qgrid2, args=args, method = 'thresholding')

2048_B8_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Mar-24 ...
2048_B8_masked.h5 copy is done
2048_B8_masked.h5 Loading data into a numpy array started 
2048_B8_masked.h5 Loading data into a numpy array finished in 64.95390291700005 seconds
2048_B8_masked.h5 Patching Started 
2048_B8_masked.h5 Patching finished in 45.25096725000003 seconds
2048_B8_masked.h5 patched pilW2_image dataset creation staring ... 
2048_B8_masked.h5 patched pilW2_image dataset creation finished in 114.97170037499995 seconds
Circular averaging starts now ... 
data received: sn=2048_B8, fr1=0
data received: sn=2048_B8, fr1=617
data received: sn=2048_B8, fr1=1234
data received: sn=2048_B8, fr1=1851
data received: sn=2048_B8, fr1=2468
data received: sn=2048_B8, fr1=3085
data received: sn=2048_B8, fr1=3702
data received: sn=2048_B8, fr1=4319
2048_B8_masked.h5 total 1-d averaging time 541.735027042 seconds


In [7]:
############### -------- configure patch size and copy the pathces tuplte to the next cell for computing 1-d averaging for all frames ------------- ################### (Independent Cell)
%matplotlib widget
frame = 2223         # possible_bkg frame
file = '2048_B8.h5'
patches = patches = [
                             [[800, 0], [960, 120]],
                             [[0, 0], [960, 120]],
                             [[590, 220], [610, 240]],
                             [[0, 590], [45, 615]],
                             [[315, 740], [345, 765]],
                             [[630, 900], [655, 920]],
                             [[20, 955], [70, 985]],
                             [[890, 375], [920, 400]],
                             [[865, 990], [985, 1030]],
                             [[310, 420], [330, 435]]
                    ]

patching(file, frame, qgrid2, args=patches, method = 'rec_circ_patch');

frame information extraction completes with _SAXS shape (1, 1043, 981) _WAXS shape (1, 1043, 981)...
Circular averaging starts now ... 
2048_B8_masked_2223.h5 total 1-d averaging time 0.44504058299992266 seconds


/Users/bashit.a/Documents/Alzheimer/Codes/py4xs/data2d.py:350: RuntimeWarning: invalid value encountered in sqrt
  dI = np.sqrt(Iq2-Iq*Iq)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# 1-d averaging after rectangular and/or circular patches for one file  (Independent Cell)
### spec
source_file = "2048_B8.h5"
patches = [
#                              [[800, 0], [980, 150]],
#                              [[245, 0], [385, 130]],
          ]

### computation
masked_file = circ_avg_from_patches(source_file, qgrid2, patches)

2302_EC-roi2_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Mar-24/Sample-2 ...
2302_EC-roi2_masked.h5 copy is done
2302_EC-roi2_masked.h5 Loading data into a numpy array started 
2302_EC-roi2_masked.h5 Loading data into a numpy array finished in 56.88684691599997 seconds
2302_EC-roi2_masked.h5 Patching Started 
2302_EC-roi2_masked.h5 Patching Finished
2302_EC-roi2_masked.h5 patched pilW2_image dataset creation staring ... 
2302_EC-roi2_masked.h5 patched pilW2_image dataset creation finished in 90.95033787500006 seconds
Circular averaging starts now ... 
data received: sn=2302_EC-roi2, fr1=0
data received: sn=2302_EC-roi2, fr1=480
data received: sn=2302_EC-roi2, fr1=960
data received: sn=2302_EC-roi2, fr1=1440
data received: sn=2302_EC-roi2, fr1=1920
data received: sn=2302_EC-roi2, fr1=2400
data received: sn=2302_EC-roi2, fr1=2880
data received: sn=2302_EC-roi2, fr1=3360
2302_EC-roi2_masked.h5 total 1-d averaging time 482.8924838329999 seconds
Successfully created 23

In [18]:
## patching information retrival from _masked.h5 file    (Independent Cell)
#specs
file = "2048_B8_masked.h5"
# computation
try: patches = get_patch_attributes(f'{file}'); print(patches)
except: print(f'{file} does not have patches information on the processed folder')

[0, 30000, 16000]


In [19]:
### check config.json file integrity     (Independent Cell)
No_error_no_prb = get_json_str_data('config-dec-15.json');

In [3]:
# 1-d averaging after patches for ALL files in the current directory   (Independent Cell)

data = get_json_str_data("config-dec-15.json")
### do masking and 1-d averaging (Lin Yang's code by BNL used here for 1-d averaging)
files = []
for file in data['files']:
    source_file , patches = file['name'], file['patches']
    try: masked_file = circ_avg_from_patches(source_file, qgrid2, patches)
    except: continue
    files.append((source_file, masked_file))

print(f'Successfully created (source file, masked file) \n\n')
[print(s, '--->' ,t) for s,t in files]
pass

1934_B8_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Dec-15 ...
1934_B8_masked.h5 copy is done
1934_B8_masked.h5 Loading data into a numpy array started 
1934_B8_masked.h5 Loading data into a numpy array finished in 64.04968554099923 seconds
1934_B8_masked.h5 Patching Started 
1934_B8_masked.h5 Patching Finished
1934_B8_masked.h5 patched pilW2_image dataset creation staring ... 
1934_B8_masked.h5 patched pilW2_image dataset creation finished in 97.23200600000018 seconds
Circular averaging starts now ... 
data received: sn=1934_B8, fr1=0
data received: sn=1934_B8, fr1=517
data received: sn=1934_B8, fr1=1034
data received: sn=1934_B8, fr1=1551
data received: sn=1934_B8, fr1=2068
data received: sn=1934_B8, fr1=2585
data received: sn=1934_B8, fr1=3102
data received: sn=1934_B8, fr1=3619
1934_B8_masked.h5 total 1-d averaging time 461.931534542 seconds
DID not PATCH 1934_SN_masked.h5 already exists - did not copy, 
1-D Averaging is NOT performed as 1934_SN_masked.h5 alre

In [19]:
### 1d circularly averaged data for masked file 
%matplotlib widget
import ipywidgets   # from IPython.display import display; display(amp)

# specs 
source_file = "2048_B8.h5"          # None to avoid plotting
masked_file = "2048_B8_masked.h5"   # None to avoid plotting
scattering = '_WAXS2'

# semi-specs
file = masked_file 

# computation
if source_file is not None:
    Iq_S = read_Iq(source_file, scattering)     
if masked_file is not None:
    Iq_M = read_Iq(masked_file, scattering)     
    
Width, Height = width_height(file)    # considering the fact that width and height is same for source and masked file
n_patterns = Width*Height 

f,ax = plt.subplots()
def update_plot(frame):
    ax.clear()
    ax.set_yscale('log')
    if source_file is not None:
        ax.plot(qgrid2, Iq_S[frame], label = f'Source - {frame}')
    if masked_file is not None:
        ax.plot(qgrid2, Iq_M[frame], label = f'Masked - {frame}')
    ax.legend()

frame = ipywidgets.IntSlider(min=0, max=n_patterns-1, value=0, description = "Frame")
ipywidgets.interactive(update_plot, frame=frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Frame', max=4940), Output()), _dom_classes=('widget-inte…

In [20]:
### show heat map for any file
%matplotlib widget

file = '2048_B8_masked.h5'
#file = masked_file    
scatterings = ('_SAXS', '_WAXS2')

#plot_one_heat_maps(file, scatterings)
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)
print('Task Finished. Figure Number = ' , f.number)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
Task Finished. Figure Number =  1


In [21]:
################# --------- Find Q range to plot heat maps --------- #################
%matplotlib widget
import ipywidgets         # from IPython.display import display; display(amp)
from ipywidgets import fixed

# semi-specs
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

### q_low, q_high, q_low - idx[2] and q_high + idx[2] for poly fit, number of points
indices = ((0.115, 0.16, 13,   48, '_SAXS' , 'bumpy'),      # 72, 81
           (1.48,  1.57, 10, 4.15, '_WAXS2', 'paraffin'),   # 345, 355
           (1.30,  1.38, 10,  4.7, '_WAXS2', 'amyloid'),    # 309, 323
           (1.89,  1.93,  7,  3.2, '_WAXS2', 'mica')        # 427, 433
          )

## polyhit with heatmap from drop down list
files = cwd_files_search_with('.h5')
dropdown = ipywidgets.Dropdown(options=files, value = files[0],description='File : ', disabled=False)
ipywidgets.interact(file_polyfit_heatmap_plot, file = dropdown, indices = fixed(indices), qgrid2 = fixed(qgrid2))

## polyhit with heatmap for one file
# file = '2048_B8.h5'
# file_polyfit_heatmap_plot(file, indices, qgrid2)

interactive(children=(Dropdown(description='File : ', options=('2048_B8.h5', '2048_B8_masked.h5', 'exp.h5'), v…

<function essential_func.file_polyfit_heatmap_plot(file, indices, qgrid2)>

In [2]:
### Merge all PDFs in the directory specified
### you can put all your pdfs that you want to merge to the PDF folder
from essential_func import *
pdfs_merging(directory = '/PDF', output = 'result.pdf')

Current location /Users/bashit.a/Documents/Alzheimer/Dec-15
Changing to /Users/bashit.a/Documents/Alzheimer/Dec-15//PDF directory 
result.pdf file written successfully
Back to root directory  /Users/bashit.a/Documents/Alzheimer/Dec-15


In [22]:
%matplotlib widget
from essential_func import *
from py4xs.hdf import h5xs
from py4xs.slnxs import trans_mode

print(f'{os.getcwd()}')

# specs
# file = '1934_SN_masked.h5'  
# frames = (4518, 3687, 3045)     # amyloid, mica, dark  ---> frames can be choosen from SAXS heat map

file = '2048_B8_masked.h5'
frames = (2080, 1358, 2223, )     # amyloid, mica, dark
show_eb = False                  # show errorbar in the plot

## semi-spec
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# creating objects
dt  = h5xs(f'{file}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file)
dt.load_d1s(sn=sn)

## heat map from file
scatterings = ('_SAXS', '_WAXS2')
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)

### 1-d plot for bright, moderate and dark frames
scatterings = ('_SAXS',          '_WAXS2',        'merged',    )
colors =      (('r','b','g'), ('r','b','g'), ('r+','bo','g--'),)
f, axs = plt.subplots(1, 3, figsize=(15,5), num=f'{file}')
xscale = 'linear'
yscale = 'log'

for i, (scattering, color) in enumerate(zip(scatterings,colors)):
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[0]].data, color[0], label = f'bright {frames[0]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[1]].data, color[1], label = f'moderate {frames[1]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[2]].data, color[2], label = f'dark {frames[2]}')
    axs[i].set(title = f'{scattering} {file}', xscale= xscale, yscale= yscale)
    axs[i].legend()

## background correction calculations
sc_factor=1.

dset = dt.d1s[sn][scattering][frames[0]]
dbak = dt.d1s[sn][scattering][frames[2]]

dt.set_trans(transMode=trans_mode.external)
sc  = dset.trans / dbak.trans
print(f'Scaling factor, {dset.trans}/{dbak.trans} = {sc}')
Iq  = dset.data - dbak.data * sc * sc_factor
dI  = dset.err  + dbak.err * sc * sc_factor
idx = (dset.data > 0) & (dbak.data > 0)

## plotting settings
f, axs = plt.subplots(1,2,figsize=(16,4))
plt.subplots_adjust(bottom=0.15)

axs[0].plot(dbak.qgrid[idx], dset.data[idx], color[0], label = f'bright {frames[0]}')
axs[0].plot(dbak.qgrid[idx], dbak.data[idx], color[2], label = f'dark-bcg {frames[2]}')

if show_eb:
    axs[0].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx])
    axs[1].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx], label = f'{frames[0]} bcor')
else:
    axs[0].plot(dbak.qgrid[idx], Iq[idx], label = 'bcor')
    axs[1].plot(dbak.qgrid[idx], Iq[idx], label = f'{frames[0]} bcor')


axs[0].set(xlabel = "$q (\AA^{-1})$", ylabel = "$I$", title = f'Background correction {file} errorbar = {show_eb}', xscale = xscale, yscale = yscale, )
axs[1].set(xscale = xscale, yscale = yscale, title = f'Final Background Subtracted {file} errorbar = {show_eb}' , )
axs[0].legend()
axs[1].legend()
print(Iq[1:10])

/Users/bashit.a/Documents/Alzheimer/Mar-24


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Scaling factor, 45131.328/45333.208 = 0.9955467523939625


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[  0.         527.69596259 355.4387128  240.5995019  169.68693625
 123.96353014  93.57119693  72.89987768  56.8460686 ]


In [3]:
### learn this part of the code
def flatten(x):
    """
    l = [2,[[1,2]],1]
    list(flatten(l))
    """
    for item in x:
        if type(item) in [list]:
            for num in flatten(item):
                yield(num)
        else:
            yield(item)

flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
l = [2,[[1,2]],1]
flatten(l)

[2, 1, 2, 1]

In [28]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np


class Cursor:
    def __init__(self, ax):
        self.ax = ax
        self.lx = ax.axhline(color='k')  # the horiz line
        self.ly = ax.axvline(color='k')  # the vert line

        # text location in axes coords
        self.txt = ax.text(0.7, 0.9, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x, y = event.xdata, event.ydata
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)

        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()


class SnaptoCursor:
    """
    Like Cursor but the crosshair snaps to the nearest x, y point.
    For simplicity, this assumes that *x* is sorted.
    """

    def __init__(self, ax, x, y):
        self.ax = ax
        self.lx = ax.axhline(color='k')  # the horiz line
        self.ly = ax.axvline(color='k')  # the vert line
        self.x = x
        self.y = y
        # text location in axes coords
        self.txt = ax.text(0.7, 0.9, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x, y = event.xdata, event.ydata
        indx = min(np.searchsorted(self.x, x), len(self.x) - 1)
        x = self.x[indx]
        y = self.y[indx]
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)

        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        print('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()


t = np.arange(0.0, 1.0, 0.01)
s = np.sin(2 * 2 * np.pi * t)

fig, ax = plt.subplots()
ax.plot(t, s, 'o')
cursor = Cursor(ax)
fig.canvas.mpl_connect('motion_notify_event', cursor.mouse_move)

fig, ax = plt.subplots()
ax.plot(t, s, 'o')
snap_cursor = SnaptoCursor(ax, t, s)
fig.canvas.mpl_connect('motion_notify_event', snap_cursor.mouse_move)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
%matplotlib widget
import seaborn as sns
df = sns.load_dataset('iris')
sns.pairplot(df,kind='reg',) #markers=["o","s","D"])
plt.show()
print(df)
plt.figure()
plt.scatter(df['sepal_width'], df['sepal_length'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

     sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
### -------- py4xs version --------

%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
# Fixing random state for reproducibility
np.random.seed(19680801)

Height = 3
Width = 6
X = np.random.randint(1, 20, size=(Height, Width))

fig, ax = plt.subplots()
ax.imshow(X, interpolation = 'none', origin='upper', extent=[0,Width,0,Height],aspect='equal')

def snaking( Width, Height, X=None,):
    if X is None:
        X_coor = np.arange(Height*Width).reshape(Height,Width)
    else:
        X_coor = X
    output = np.zeros_like(X_coor)           # 
    temp = np.flipud(X_coor)                # flipud matrix
    idx_odd = np.arange(1,Height,2)                # Get odd indices to fliplr
    output[idx_odd,:]  = np.fliplr(temp[idx_odd])   # flipping odd indices
    idx_even = np.arange(0,Height,2)               # Get odd indices to leave to as it is
    output[idx_even,:] = temp[idx_even]             # leaving as it is
    return output

frame_cor = snaking(Width, Height)
diff_patterns = snaking(Width, Height, X)

print('Diff_patt_idx_before snaking: \n', np.arange(Height*Width).reshape(Height,Width), '\n\n',
      'Diff_patt_before snaking: \n', X, '\n\n', 
      'X,Y coordinates_after_snaking: \n', frame_cor, '\n\n',
      'Diff_patt_after snaking: \n', diff_patterns, '\n\n',)

numrows, numcols = X.shape
def format_coord(x, y):
    col = int(x)
    row = int(y)
    if 0 <= col < numcols and 0 <= row < numrows:
        z = np.flipud(frame_cor)[row, col]
        return 'x=%1.4f, y=%1.4f, FRAME=%d' % (x, y, z)
    else:
        return 'x=%1.4f, y=%1.4f' % (x, y)

ax.format_coord = format_coord

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Diff_patt_idx_before snaking: 
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]] 

 Diff_patt_before snaking: 
 [[13 14  7  3  3  4]
 [18  5 11 19 12  9]
 [ 6 12  4  6 17 10]] 

 X,Y coordinates_after_snaking: 
 [[12 13 14 15 16 17]
 [11 10  9  8  7  6]
 [ 0  1  2  3  4  5]] 

 Diff_patt_after snaking: 
 [[ 6 12  4  6 17 10]
 [ 9 12 19 11  5 18]
 [13 14  7  3  3  4]] 




In [19]:
### Finding Radius of gyration
from py4xs.hdf import h5xs,h5exp,lsh5,proc_d1merge
from py4xs.data2d import Data2d
from py4xs.slnxs import trans_mode
import numpy as np
import pylab as plt
import warnings,json

warnings.filterwarnings('ignore')

dt0  = h5xs("mica.h5", transField="em2_sum_all_mean_value")
dt  = h5xs("2048_B8.h5", transField="em2_sum_all_mean_value")

dt.load_d1s()
dt0.load_d1s()

dt.set_trans(transMode=trans_mode.external)
dt0.set_trans(transMode=trans_mode.external)

print(dt.d1s['2048_B8']['merged'][2080].trans)
print(dt0.d1s['mica']['merged'][10].trans)
#print(dt.d1s['2048_B8']['merged'][2080].overlaps)
#dt0.d1s['mica']['merged'][10].plot()

45131.328
45639.848


In [20]:
td = np.vstack((dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data))
qs = 0.007
qe = 10
rg = 15
# td = td[:, td[0, :] >= qs]     # find td for  greater than q=0.007
# td = td[:, td[0, :] <= qe]     # find td for smaller than q=1.3/1.5=0.086
# td[0, :] = td[0, :] * td[0, :]
# td[1, :] = np.log(td[1, :])
# rg, i0 = np.polyfit(td[0, :], td[1, :], 1)

In [21]:
from numpy.polynomial import polynomial as P
cnt = 0
while cnt < 10:
    if qe > 1.3/rg and 1.3/rg > qs+0.004: 
        qe = 1.3/rg
    td = np.vstack((dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data))
    td = td[:, td[0, :] >= qs]     # find td for  greater than q=0.007
    td = td[:, td[0, :] <= qe]     # find td for smaller than q=1.3/1.5=0.086
    td[0, :] = td[0, :] * td[0, :]
    td[1, :] = np.log(td[1, :])
    rg, i0 = np.polyfit(td[0, :], td[1, :], 1)
    i0 = np.exp(i0)
    
    print(cnt, qe, rg, i0)
    
    if rg<0:
        rg = np.sqrt(-rg * 3.)
    else:
        rg = 1e-6   # 
        print("likely strong inter-particle interaction ...")
        break
    cnt += 1
print(rg)

0 0.08666666666666667 -831.264016704801 20.377926211495332
1 0.026032341584365102 -6234.912831830037 281.9576616244667
2 0.026032341584365102 -6234.912831830037 281.9576616244667
3 0.026032341584365102 -6234.912831830037 281.9576616244667
4 0.026032341584365102 -6234.912831830037 281.9576616244667
5 0.026032341584365102 -6234.912831830037 281.9576616244667
6 0.026032341584365102 -6234.912831830037 281.9576616244667
7 0.026032341584365102 -6234.912831830037 281.9576616244667
8 0.026032341584365102 -6234.912831830037 281.9576616244667
9 0.026032341584365102 -6234.912831830037 281.9576616244667
136.76526786977064


In [72]:
%matplotlib widget
qmax = 5;
dmax=200.;

if dt.qgrid[-1] < qmax: 
    qmax = dt.qgrid[-1]

tqgrid = np.arange(0, qmax, qmax / len(dt.qgrid))
tint = np.interp(tqgrid, dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data)
print(tint.shape)
tint[tqgrid * rg < 1.] = i0 * np.exp(-(tqgrid[tqgrid * rg < 1.] * rg) ** 2 / 3.)
#print(tint)

# tint -= tint[-10:].sum()/10
# Hanning window for reducing fringes in p(r)
tw = np.hanning(2 * len(tqgrid) + 1)[len(tqgrid):-1]

# plotting hanning window
tmp = np.hanning(2 * len(tqgrid) + 1)
#plt.plot(np.arange(0,len(tint[tqgrid * rg < 1.])), tint[tqgrid * rg < 1.])
#plt.yscale('log')
tint *= tw
#plt.show()

trgrid = np.arange(0, dmax, 1.)
kern = np.asmatrix([[rj ** 2 * np.sinc(qi * rj / np.pi) for rj in trgrid] for qi in tqgrid])
tt = np.asmatrix(tint * tqgrid ** 2).T
tpr = np.reshape(np.array((kern.T * tt).T), len(trgrid))
tpr /= tpr.sum()
print(tpr.shape)

(690,)
(200,)


In [39]:
tqgrid = np.arange(0, qmax, qmax / len(dt.qgrid))
tint = np.interp(tqgrid, dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data)
tqgrid[0:10]

array([0.        , 0.00463043, 0.00926087, 0.0138913 , 0.01852174,
       0.02315217, 0.02778261, 0.03241304, 0.03704348, 0.04167391])